# STATISTICS TIME in queries

A friend is writing a training course on how the command `STATISTICS TIME` can be used to tune query performance and asks for your help to complete a presentation. He requires two queries that return NBA team details where the host city had a 2017 population of more than two million.

NBA team details can be queried from the` NBA Season 2017-2018` database and city populations can be queried by adding in tables from the `Earthquakes` database.

Each query uses a different filter on the Teams table.

- Query 1
    - Filters the Teams table using IN and three sub-queries
- Query 2
    - Filters the Teams table using EXISTS

```
SET STATISTICS TIME ON -- Turn the time command on

-- Query 1
SELECT * 
FROM Teams
-- Sub-query 1
WHERE City IN -- Sub-query filter operator
      (SELECT CityName 
       FROM Cities) -- Table from Earthquakes database
-- Sub-query 2
   AND City IN -- Sub-query filter operator
	   (SELECT CityName 
	    FROM Cities
		WHERE CountryCode IN ('US','CA'))
-- Sub-query 3
    AND City IN -- Sub-query filter operator
        (SELECT CityName 
         FROM Cities
	     WHERE Pop2017 >2000000);

-- Query 2
SELECT * 
FROM Teams AS t
WHERE EXISTS -- Sub-query filter operator
	(SELECT 1 
     FROM Cities AS c
     WHERE t.City = c.CityName -- Columns being compared
        AND c.CountryCode IN ('US','CA')
          AND c.Pop2017 > 2000000);

SET STATISTICS TIME OFF -- Turn the time command off                   
```

# STATISTICS TIME results

In the previous exercise, the `STATISTICS TIME` command was used on two different queries. The following table summarizes an analysis of the elapsed time statistics for each query.
```
Query	        Details                                        Average elapsed time (ms)
1	    Filters the Teams table using IN and three sub-queries	    20
2	    Filters the Teams table using EXISTS	                    3
```
What conclusion can you make from this summary?

- The second query that uses `EXISTS` is more efficient.

# STATISTICS IO: Example 1

Your sales company has just taken on a new regional manager for Western Europe. He has asked you to provide him daily updates on orders shipped to some key Western Europe capital cities. As this data is time sensitive, you want a robust query that is tuned to return the results as quickly as possible.

You initially decide on a query that uses two sub-queries: one in the `SELECT` statement to get the count of orders and one using a filter condition with an `IN` operator.

You will turn on the `STATISTICS IO` command to review the page read statistics.

```
SET STATISTICS IO ON -- Turn the IO command on


-- Example 1
SELECT CustomerID,
       CompanyName,
       (SELECT COUNT(*) 
	    FROM Orders AS o -- Add table
		WHERE c.CustomerID = o.CustomerID) CountOrders
FROM Customers AS c
WHERE CustomerID IN -- Add filter operator
       (SELECT CustomerID 
	    FROM Orders 
		WHERE ShipCity IN
            ('Berlin','Bern','Bruxelles','Helsinki',
			'Lisboa','Madrid','Paris','London'));

SET STATISTICS IO OFF -- Turn the IO command off

```



From the `STATISTICS IO` output below, how many data pages from the Orders table were read from memory to complete the query?
```
Table 'Customers'. Scan count 1, logical reads 2, physical reads 0,...
Table 'Orders'. Scan count 2, logical reads 32, physical reads 0,...
```
- Thirty-two

# STATISTICS IO: Example 2

In the previous exercise, you were asked you to provide a new regional manager daily updates on orders shipped to Western Europe capital cities. You initially created a query that contained two sub-queries. You decide to do a rewrite and use an `INNER JOIN`.

The `STATISTICS IO` command is turned on. You will need to turn it off after completing the query.

```
SET STATISTICS IO ON -- Turn the IO command on

-- Example 2
SELECT c.CustomerID,
       c.CompanyName,
       COUNT(o.CustomerID)
FROM Customers AS c
INNER JOIN Orders AS o -- Join operator
    ON c.CustomerID = o.CustomerID
WHERE o.ShipCity IN -- Shipping destination column
     ('Berlin','Bern','Bruxelles','Helsinki',
	 'Lisboa','Madrid','Paris','London')
GROUP BY c.CustomerID,
         c.CompanyName;


SET STATISTICS IO OFF -- Turn the IO command off
```

From the `STATISTICS IO` output below, how many data pages from the Orders table were read from memory to complete the query?
```
Table 'Customers'. Scan count 1, logical reads 2, physical reads 0,...
Table 'Orders'. Scan count 2, logical reads 16, physical reads 0,...
```
- Sixteen

# STATISTICS IO comparison

Using the `STATISTICS IO` outputs from the example queries in the previous two exercises, what might you conclude?

Example 1
```
Table 'Customers'. Scan count 1, logical reads 2, physical reads 0,....
Table 'Orders'. Scan count 2, logical reads 32, physical reads 0,...
```
Example 2
```
Table 'Customers'. Scan count 1, logical reads 2, physical reads 0,....
Table 'Orders'. Scan count 2, logical reads 16, physical reads 0,...
```

- The Example 2 query will run faster than the Example 1 query.

# Test your knowledge of indexes

Which of the following statements about indexes is FALSE?

- Clustered and nonclustered indexes are applied to table rows.

# Clustered index

Clustered indexes can be added to tables to speed up search operations in queries. You have two copies of the `Cities` table from the `Earthquakes` database: one copy has a clustered index on the `CountryCode` column. The other is not indexed.

You have a query on each table with a different filter condition:

- Query 1
    - Returns all rows where the country is either `Russia` or `China`.
- Query 2
    - Returns all rows where the country is either `Jamaica` or `New Zealand`.

```
-- Query 1
SELECT *
FROM Cities
WHERE CountryCode = 'RU' -- Country code
		OR CountryCode = 'CN' -- Country code
```

```
-- Query 2
SELECT *
FROM Cities
WHERE CountryCode IN ('JM','NZ') -- Country codes
```

For these two queries, what conclusion could you make using the following output from the `STATISTICS IO` command?

```
4694 results returned
Table 'Cities'. ..., logical reads 274, ... ,
```

```
212 results returned
Table 'Cities'. ..., logical reads 10, ... ,
```
- Query 2 accesses a clustered index because logical reads indicates fewer data pages were accessed compared to Query 1

# Sort operator in execution plans

Execution plans can tell us if and where a query used an internal sorting operation. Internal sorting is often required when using an operator in a query that checks for and removes duplicate rows.

You are given an execution plan of a query that returns all cities listed in the `Earthquakes` database. The query appends queries from the `Nations` and `Cities` tables. Use the following execution plan to determine if the appending operator used is `UNION` or `UNION ALL`
<center><img src="images/04.12.bmp"  style="width: 400px, height: 300px;"/></center>


```
SELECT CityName AS NearCityName,
	   CountryCode
FROM Cities

UNION -- Append queries

SELECT Capital AS NearCityName,
       Code2 AS CountryCode
FROM Nations;
```

# Test your knowledge of execution plans

Which one of the following is NOT information you can get from an execution plan in SQL Server?

- The total duration of the query, in milliseconds, from execution to returning the complete results